## Advent of code 2022 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers
#import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines, nostrip=False):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end,
    by default also strips all lines (if nostrip is set only strips empty lines)'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line_str=line.strip()
        if nostrip==False or len(line_str)<1:
            line=line_str
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2022 day 10
# mv ~/Downloads/input* data_src/2022-day-10-input.txt
# big input file looks like: list of instructions
# idea: part 1 parse as tuples, then simulate
# part 2: same

sample2='''
noop
addx 3
addx -5
''' # part 1

sample3='''
addx 15
addx -11
addx 6
addx -3
addx 5
addx -1
addx -8
addx 13
addx 4
noop
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx -35
addx 1
addx 24
addx -19
addx 1
addx 16
addx -11
noop
noop
addx 21
addx -15
noop
noop
addx -3
addx 9
addx 1
addx -3
addx 8
addx 1
addx 5
noop
noop
noop
noop
noop
addx -36
noop
addx 1
addx 7
noop
noop
noop
addx 2
addx 6
noop
noop
noop
noop
noop
addx 1
noop
noop
addx 7
addx 1
noop
addx -13
addx 13
addx 7
noop
addx 1
addx -33
noop
noop
noop
addx 2
noop
noop
noop
addx 8
noop
addx -1
addx 2
addx 1
noop
addx 17
addx -9
addx 1
addx 1
addx -3
addx 11
noop
noop
addx 1
noop
addx 1
noop
noop
addx -13
addx -19
addx 1
addx 3
addx 26
addx -30
addx 12
addx -1
addx 3
addx 1
noop
noop
noop
addx -9
addx 18
addx 1
addx 2
noop
noop
addx 9
noop
noop
noop
addx -1
addx 2
addx -37
addx 1
addx 3
noop
addx 15
addx -21
addx 22
addx -6
addx 1
noop
addx 2
addx 1
noop
addx -10
noop
noop
addx 20
addx 1
addx 2
addx 2
addx -6
addx -11
noop
noop
noop
''' # part 1

class Context1:
    def __init__(self):
        self.totalsig=0

    def do_cycle(self, cycle, x):
        if (cycle-20)%40==0 and 20<=cycle<=220:
            self.totalsig+= x*cycle

def do_cycles(data, ctx):
    x=1
    cycle=1
    for tup in data:
        if tup[0]=='noop':
            ctx.do_cycle(cycle, x)
            cycle+=1
        elif tup[0]=='addx':
            ctx.do_cycle(cycle, x)
            cycle+=1
            ctx.do_cycle(cycle, x)
            cycle+=1            
            x+=tup[1]
        else:
            assert False

class Context2:
    def __init__(self):
        self.row=''
        self.board=[]

    def do_cycle(self, cycle, x):
        cyc2=((cycle-1)%40)+1
        c='#' if x<=cyc2<=x+2 else ' '
        self.row+=c
        if len(self.row)>=40:
            self.board.append(self.row)
            self.row=''

sample1=open('data_src/2022-day-10-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ s.split() for s in lines ]
for tup in data:
    if len(tup)>1:
        tup[1]=int(tup[1])
ctx=Context1()
do_cycles(data, ctx)
print(f'part 1: {ctx.totalsig}')
ctx=Context2()
do_cycles(data, ctx)
print('part 2:')
for row in ctx.board:
    print(row)

# part 1: 14420
# part 2: RGLRBZAU

In [ ]:
# 2022 day 9
# mv ~/Downloads/input* data_src/2022-day-9-input.txt
# big input file looks like: long list of instructions
# idea: part 1 parse as rows of pairs, then simulate according to instructions,
#  pretty easy using dx and dy to model movement of the tail
# for part 2: there's one head and 9 tails, each in a head-tail relationship with each other,
# the last one is the real tail, so keep tails in a list and after each movement of the head simply
# have each tail follow the knot in front

sample2='''
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
''' # part 1

sample3='''
R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20
''' # part 2

def sign(n):
    if n<0:
        return -1
    elif n==0:
        return 0
    else:
        return 1

def update_tail(hx, hy, tx, ty):
    if abs(hx-tx)<2 and abs(hy-ty)<2:
        return tx,ty
    dx=0
    dy=0
    if tx==hx:
        dy=sign(hy-ty)
    elif ty==hy:
        dx=sign(hx-tx)
    else:
        dy=sign(hy-ty)
        dx=sign(hx-tx)
    ntx=tx+dx
    nty=ty+dy
    return ntx,nty

def count_visited1(data):
    '''one head, one tail'''
    tvisited=set()
    hx=0
    hy=0
    tx=0
    ty=0
    tvisited.add( (tx,ty) )
    for row in data:
        direc,sn=row
        n=int(sn)
        for _ in range(n):
            if direc=='R':
                hx+=1
            elif direc=='L':
                hx-=1
            elif direc=='U':
                hy-=1
            elif direc=='D':
                hy+=1
            else:
                assert False
            tx,ty=update_tail(hx, hy, tx, ty)
            tvisited.add( (tx,ty) )
    return len(tvisited)

def count_visited10(data):
    '''one head, 9 tails'''
    tvisited=set()
    hx=0
    hy=0
    tails=[] # x,y of 9 tails, last is the 'real tail'
    for _ in range(9):
        tails.append( (0,0) )
    tvisited.add( (tails[-1][0],tails[-1][1]) )
    for row in data:
        direc,sn=row
        n=int(sn)
        for _ in range(n):
            if direc=='R':
                hx+=1
            elif direc=='L':
                hx-=1
            elif direc=='U':
                hy-=1
            elif direc=='D':
                hy+=1
            else:
                assert False
            for i,tup in enumerate(tails):
                tx,ty=tup
                if i==0:
                    tx,ty=update_tail(hx, hy, tx, ty)
                else:
                    tx,ty=update_tail(tails[i-1][0], tails[i-1][1], tx, ty)
                tails[i]=(tx,ty)
            tvisited.add( (tails[-1][0],tails[-1][1]) )
    return len(tvisited)

sample1=open('data_src/2022-day-9-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ s.split() for s in lines ]
c1=count_visited1(data)
print(f'part 1: {c1}')
c2=count_visited10(data)
print(f'part 2: {c2}')

# part 1: 6494
# part 2: 2691

In [ ]:
# 2022 day 8
# mv ~/Downloads/input* data_src/2022-day-8-input.txt
# big input file looks like: block of 99 lines

sample2='''
30373
25512
65332
33549
35390
'''

def set_visible(board):
    reached={} # maps (x,y) to visible boolean
    for y in range(len(board)):
        maxh=None
        for x in range(len(board[0])):
            h=int(board[y][x])
            if maxh is None or h>maxh:
                reached[ (x,y) ]=True
                maxh=h
        maxh=None
        for x in range(len(board[0])-1, -1, -1):
            h=int(board[y][x])
            if maxh is None or h>maxh:
                reached[ (x,y) ]=True
                maxh=h
    for x in range(len(board[0])):
        maxh=None
        for y in range(len(board)):
            h=int(board[y][x])
            if maxh is None or h>maxh:
                reached[ (x,y) ]=True
                maxh=h
        maxh=None
        for y in range(len(board)-1, -1, -1):
            h=int(board[y][x])
            if maxh is None or h>maxh:
                reached[ (x,y) ]=True
                maxh=h
    return reached

def get_viewdist(h, maxh, ed):
    '''current tree at height h, maxh maps height to last edge distance, ed is current edge distance'''
    if ed==0:
        maxh[h]=ed
        return 0
    lastblock=None # max ed of tree at least as high as this one
    for prevh,preved in maxh.items():
        if prevh>=h and (lastblock is None or preved>lastblock):
            lastblock=preved
    maxh[h]=ed
    if lastblock is None:
        return ed
    else:
        return ed-lastblock

def scenic_score(board):
    score={} # maps (x,y) to score
    for y in range(len(board)):
        maxh={} # maps height to last position
        ed=0
        for x in range(len(board[0])):
            score[(x,y)]=1
            h=int(board[y][x])
            vd=get_viewdist(h, maxh, ed)
            score[(x,y)]=score[(x,y)]*vd
            ed+=1
        maxh={}
        ed=0
        for x in range(len(board[0])-1, -1, -1):
            h=int(board[y][x])
            vd=get_viewdist(h, maxh, ed)
            score[(x,y)]=score[(x,y)]*vd
            ed+=1
    for x in range(len(board[0])):
        maxh={}
        ed=0
        for y in range(len(board)):
            h=int(board[y][x])
            vd=get_viewdist(h, maxh, ed)
            score[(x,y)]=score[(x,y)]*vd
            ed+=1
        maxh={}
        ed=0
        for y in range(len(board)-1, -1, -1):
            h=int(board[y][x])
            vd=get_viewdist(h, maxh, ed)
            score[(x,y)]=score[(x,y)]*vd
            ed+=1
    return score

def max_scen(score):
    return max(score.values())

sample1=open('data_src/2022-day-8-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
reached=set_visible(lines)
print(f'part 1: {len(reached)}')
score=scenic_score(lines)
maxsc=max_scen(score)
print(f'part 2: {maxsc}')

# part 1: 1859
# part 2: 332640

In [ ]:
# 2022 day 7
# mv ~/Downloads/input* data_src/2022-day-7-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
'''

def parse_files(data):
    root={} # maps filenames to either size or dict for subdirs
    cwd=root
    for row in data:
        if row[0]=='$' and row[1]=='cd':
            if row[2]=='/':
                cwd=root
                continue
            elif row[2]=='..':
                cwd=cwd[row[2]]
                continue
            prev=cwd
            cwd=cwd[row[2]]
            cwd['..']=prev
        elif row[0]=='$' and row[1]=='ls':
            pass
        else:
            if row[0]=='dir':
                cwd.setdefault(row[1], {})
            else:
                cwd[row[1]]=int(row[0])
    return root

def traverse_sum_max(node, maxsz, summary=None, dirsizes=None):
    '''return totalsize, isdir'''
    if isinstance(node, int):
        return (node, False)
    elif isinstance(node, dict):
        totalsz=0
        for k, v in node.items():
            if k=='..':
                continue
            sz=traverse_sum_max(v, maxsz, summary, dirsizes)
            if sz[0]<=maxsz and sz[1] and summary is not None:
                summary[0]+=sz[0]
            if sz[1]:
                dirsizes.append( (k, sz[0]) )
            totalsz+=sz[0]
        return (totalsz, True)
    else:
        assert False

sample1=open('data_src/2022-day-7-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ s.split() for s in lines ]
root=parse_files(data)
# part 1
summary=[0]
dirsizes=[]
sz=traverse_sum_max(root, 100000, summary, dirsizes)
dirsizes.append( ('/', sz[0]) )
print(f'{sz=}, {summary=}')
print('part 1:', summary[0])
# part 2
dirsizes.sort(lambda tup: tup[1])
totaldisk=70000000
needunused=30000000
curused=sz[0]
curunused=totaldisk-curused
tofree=needunused-curunused
print(f'{tofree=}')
for tup in dirsizes:
    if tup[1]>=tofree:
        print('part 2:', tup)
        break

# part 1: 1491614
# part 2: 6400111

In [ ]:
# 2022 day 6
# mv ~/Downloads/input* data_src/2022-day-6-input.txt
# big input file looks like: single long line

sample2='''
mjqjpqmgbljsphdztnvjfqwrcgsmlb
bvwbjplbgvbhsrlpgdmjqwftvncz
nppdvjthqldpwncqszvftbrmjlhg
nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg
zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw
'''

def find_marker_pos(line, n):
    for pos in range(0, len(line)-n):
        subs=line[pos:pos+n]
        if len(set(subs))==n:
            return pos+n
    return -1

sample1=open('data_src/2022-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
print('part 1')
for line in lines:
    print(line[:10], find_marker_pos(line, 4))

print('part 2')
for line in lines:
    print(line[:10], find_marker_pos(line, 14))

# part 1: 1850
# part 2: 2823

In [ ]:
# 2022 day 5
# mv ~/Downloads/input* data_src/2022-day-5-input.txt
# big input file looks like: crates side by side, (at most 9), moving instructions
# idea for part 1: parse head of lines like a board (per column, from the bottom up),
# then execute the moves,
# for part 2: execute the moves with the 'batched' variation

sample2='''
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
'''

def parse_cols(lines):
    '''split lines into cols (already parsed crate stacks), and tail'''
    groups=get_line_groups(lines, nostrip=True)
    assert len(groups)==2
    sts=groups[0]
    tail=groups[1]
    assert len(sts)>0 and len(tail)>0
    cols={} # maps col id (num) to list of crate chars, bottom-most first
    for col in range(0, 10):
        x=col*4+1
        y=len(sts)-1
        if x>=len(sts[y]):
            continue
        assert sts[y][x]==str(col+1)
        for y in range(len(sts)-2, -1, -1):
            if x<len(sts[y]) and 'A'<=sts[y][x]<='Z':
                cl=cols.setdefault(col+1, [])
                cl.append(sts[y][x])
    return cols, tail

def parse_moves(tail, cols, batched=False):
    '''perform moves in tail, changing cols, if batched move all crates in same order,
    return resulting message'''
    for line in tail:
        m=re.match(r'move\s*(\d+)\s*from\s*(\d+)\s*to\s*(\d+)', line)
        if m:
            n, src, dst=int(m.group(1)), int(m.group(2)), int(m.group(3))
            assert n>0
            if batched==False:
                for i in range(n):
                    assert len(cols[src])>0
                    c=cols[src].pop(-1)
                    cols[dst].append(c)
            else:
                assert len(cols[src])>=n
                si=len(cols[src])-n
                for i in range(n):
                    c=cols[src].pop(si)
                    cols[dst].append(c)
    msg1=''
    for k in sorted(cols.keys()):
        msg1+=cols[k][-1]
    return msg1

sample1=open('data_src/2022-day-5-input.txt').read()
lines=[s for s in sample1.splitlines() ]
cols, tail=parse_cols(lines)
msg1=parse_moves(tail, cols, batched=False)
print('part 1:', msg1)
cols, tail=parse_cols(lines)
msg2=parse_moves(tail, cols, batched=True)
print('part 2:', msg2)

# part 1: QNHWJVJZW
# part 2: BPCZJLFJW

In [ ]:
# 2022 day 4
# mv ~/Downloads/input* data_src/2022-day-4-input.txt

sample2='''
2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8
'''

sample1=open('data_src/2022-day-4-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ re.split(r'[,\-]', s) for s in lines ]
data=[ [ int(s) for s in row ] for row in data ]
count=0
for row in data:
    a, b, c, d=row
    assert a<=b
    assert c<=d
    if (c<=a<=b<=d) or (a<=c<=d<=b):
        count+=1
print('part 1:', count)
count=0
for row in data:
    a, b, c, d=row
    if not (b<c or d<a):
        count+=1
print('part 2:', count)

# part 1 644
# part 2 926

In [ ]:
# 2022 day 3
# mv ~/Downloads/input* data_src/2022-day-3-input.txt
# big input file looks like: list of 300 character strings

sample2='''
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
'''

sample1=open('data_src/2022-day-3-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
part1score=0
for s in lines:
    n=len(s)//2
    s1=s[:n]
    s2=s[n:]
    common=set(s1).intersection(set(s2))
    c=list(common)[0]
    score=ord(c)-ord('A')+27 if 'A'<=c<='Z' else ord(c)-ord('a')+1
    #print(s1, s2, common, score)
    part1score+=score
print('part 1', part1score)
part2score=0
for i in range(0, len(lines), 3):
    s1=lines[i]
    s2=lines[i+1]
    s3=lines[i+2]
    common=set(s1).intersection(set(s2)).intersection(set(s3))
    c=list(common)[0]
    score=ord(c)-ord('A')+27 if 'A'<=c<='Z' else ord(c)-ord('a')+1
    part2score+=score
print('part 2', part2score)

# part 1 8515
# part 2 2434

In [ ]:
# 2022 day 2
# mv ~/Downloads/input* data_src/2022-day-2-input.txt
# big input file looks like: long list of pairs of chars

sample2='''
A Y
B X
C Z
'''

sample1=open('data_src/2022-day-2-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ s.split() for s in lines ]
score=0
for tup in data:
    opp, own=tup
    ownnum={'X': 0, 'Y': 1, 'Z': 2}[own]
    score+=ownnum+1
    oppnum={'A': 0, 'B': 1, 'C': 2}[opp]
    if oppnum==ownnum:
        score+=3
    elif ownnum==(oppnum+1)%3:
        score+=6
print('part 1', score)
score=0
for tup in data:
    opp, own=tup
    oppnum={'A': 0, 'B': 1, 'C': 2}[opp]
    ownnum={'X': 0, 'Y': 1, 'Z': 2}[own]
    if ownnum==0:
        ownplay=(oppnum-1)%3
    elif ownnum==1:
        ownplay=oppnum
    else:
        ownplay=(oppnum+1)%3
    score+=ownplay+1
    score+=ownnum*3
print('part 2', score)

# part 1 13052
# part 2 13693

In [ ]:
# 2022 day 1
# mv ~/Downloads/input* data_src/2022-day-1-input.txt
# big input file looks like: long list of groups of lines / numbers

sample2='''
1000
2000
3000

4000

5000
6000

7000
8000
9000

10000
'''

sample1=open('data_src/2022-day-1-input.txt').read()
groups=get_line_groups(sample1.splitlines())
groups=[ sum([int(s) for s in g]) for g in groups]
print(f'part 1: {max(groups)}')
sum3=sum(sorted(groups)[-3:])
print(f'part 2: {sum3}')

#part 1: 72511
#part 2: 212117

In [ ]:
# TEMPLATE
# 2022 day 1
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2022-day-1-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2022-day-1-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
groups=get_line_groups(sample2.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed